In [1]:
import io
from google.cloud import vision
from google.cloud.vision import types

from tqdm import tqdm
import statistics

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
def get_image(file):
    img = cv2.imread(file)
    if img.shape[0] < img.shape[1]:
        img = np.rot90(np.rot90(np.rot90(img)))
        cv2.imwrite("./rtmp.jpg", img)
        img = cv2.imread("./rtmp.jpg")
    return img

In [4]:
original = get_image("IMG_2180.JPG")

In [9]:
img = original.copy()
img = img[:, 0:img.shape[1] // 3]

ret, thresh = cv2.threshold(img[:,:,1], 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
ret, thresh2 = cv2.threshold(img[:,:,2], 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
thresh = cv2.bitwise_xor(thresh, thresh2)
edges = cv2.Canny(thresh,50,150,apertureSize = 3)
minLineLength = 1000
maxLineGap = 10

lines = cv2.HoughLinesP(thresh,1,np.pi/180,100,minLineLength,maxLineGap)
lines = [line for line in lines if abs(line[0][0] - line[0][2]) <= 10]

avgx = int(np.average([line[0][0] for line in lines]))
rest = original.copy()[:, int(avgx):]
img = img[:, 0:int(avgx)]

cv2.imwrite('crop.jpg',img)
cv2.imwrite('rest.jpg', rest)

True

In [10]:
client = vision.ImageAnnotatorClient()

/home/shashvat/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [17]:
def get_response(filein, othermode = False):
    with io.open(filein, 'rb') as image_file:
        image = types.Image(content = image_file.read())
    response = None
    if othermode:
        response = client.text_detection(image=image)
    else:
        response = client.document_text_detection(image=image)
    return response

In [18]:
crop_response = get_response("./crop.jpg")

In [19]:
for page in crop_response.full_text_annotation.pages:
    print("hello")
    print(page.text)

In [20]:
print(crop_response)

In [21]:
def rot_image(file):
    img = cv2.imread(file)
    if img.shape[0] < img.shape[1]:
        img = np.rot90(np.rot90(np.rot90(img)))
        cv2.imwrite(file, img)